In [1]:
DEVICE = 'cpu' # 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
import sys
sys.path.append('..')

In [3]:
import numpy as np
np.set_printoptions(linewidth=1000)

In [4]:
import clip
model, preprocess = clip.load('ViT-L/14', device=DEVICE)
model.eval();

In [5]:
from imagelib import Im

In [6]:
%load_ext rich

In [7]:
LABEL_PREFIX = 'sketch of '  # a  # an
CATEGORIES = list(map(str, range(10)))
LABEL_SUFFIX = ''  # ', a fruit' + ', wu t uope'  # ', in a grid'  # ', wu t uope'  # ', wu t uopeasdiajsi asjda isjd iojaios sf aisjdiasjdi'

In [8]:
labels = [f'{LABEL_PREFIX}{label}{LABEL_SUFFIX}' for label in CATEGORIES]

In [9]:
file = Im('../data/mnist/0/1001.png', convert=None)

---

In [10]:
%%timeit
image = preprocess(file.image).unsqueeze(0).to(DEVICE)

718 µs ± 52.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
image = preprocess(file.image).unsqueeze(0).to(DEVICE)

In [12]:
%%timeit
text = clip.tokenize(labels).to(DEVICE)

729 µs ± 3.46 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [13]:
text = clip.tokenize(labels).to(DEVICE)

In [14]:
%%timeit
image_features = model.encode_image(image)
image_features /= image_features.norm(dim=-1, keepdim=True)

532 ms ± 16.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
image_features = model.encode_image(image)
image_features /= image_features.norm(dim=-1, keepdim=True)

In [16]:
%%timeit
text_features = model.encode_text(text)
text_features /= text_features.norm(dim=-1, keepdim=True)

409 ms ± 16.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
text_features = model.encode_text(text)
text_features /= text_features.norm(dim=-1, keepdim=True)

---

In [18]:
%%timeit
(100.0 * image_features @ text_features.T).softmax(dim=-1)

28 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [19]:
(100.0 * image_features @ text_features.T).softmax(dim=-1)


tensor([[0.1757, 0.0605, 0.0129, 0.0131, 0.0444, 0.0472, 0.1335, 0.0728, 0.1679,
         0.2719]], grad_fn=<SoftmaxBackward0>)

In [20]:
%%timeit
model(image, text)[0].softmax(dim=-1)

868 ms ± 25.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
model(image, text)[0].softmax(dim=-1)


tensor([[0.1757, 0.0605, 0.0129, 0.0131, 0.0444, 0.0472, 0.1335, 0.0728, 0.1679,
         0.2719]], grad_fn=<SoftmaxBackward0>)